## Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

# ------- matrix A(Sigma) -------
def create_A_Sigma(cov: np.ndarray, cov_int: np.ndarray, p: int, index: int, type="normal") -> np.ndarray:
    row_num = num_sym(p)
    col_num = p * p
    A_Sigma = np.zeros((row_num, col_num))  # Initialize with zeros

    for l in range(p):
        for k in range(l + 1):  # k ≤ l
            row_idx = (k * p) + (l - num_sym(k))  # Compute row index
            
            for i in range(p):
                for j in range(p):
                    col_idx = i * p + j  # Compute column index

                    if (j != k) and (j != l):
                        A_Sigma[row_idx, col_idx] = 0
                    elif (j == k) and (k != l):
                        A_Sigma[row_idx, col_idx] = cov[l, i]
                    elif (j == l) and (l != k):
                        A_Sigma[row_idx, col_idx] = cov[k, i]
                    elif (j == k) and (j == l):
                        A_Sigma[row_idx, col_idx] = 2 * cov[j, i]

    if type == "hard":
        cov_int_2 = np.insert(cov_int, index, 0, axis=0)
        cov_int_3 = np.insert(cov_int_2, index, 0, axis=1)
        temp = np.zeros((row_num, col_num))  # Initialize with zeros

        for l in range(p):
            for k in range(l + 1):  # k ≤ l
                row_idx = (k * p) + (l - num_sym(k))  # Compute row index
                
                for i in range(p):
                    for j in range(p):
                        col_idx = i * p + j  # Compute column index

                        if ((j != k) and (j != l)) or (i == index) or (j == index) or (k == index) or (l == index):
                            temp[row_idx, col_idx] = 0
                        elif (j == k) and (k != l):
                            temp[row_idx, col_idx] = cov_int_3[l, i]
                        elif (j == l) and (l != k):
                            temp[row_idx, col_idx] = cov_int_3[k, i]
                        elif (j == k) and (j == l):
                            temp[row_idx, col_idx] = 2 * cov_int_3[j, i]
        A_Sigma = np.vstack((A_Sigma, temp))

    return A_Sigma

# ------- half-vectorization C -------
def create_vech_C(p: int, index: int, type="normal") -> np.ndarray:
    C = volatility_param * np.eye(p)
    vec_C = C[:, 0]
    for i in range(1, p):
        vec_C = np.hstack((vec_C, C[i:, i]))
    
    if type == "hard":
        C[index, index] = 0
        vec_C_2 = C[:, 0]
        for i in range(1, p):
            vec_C_2 = np.hstack((vec_C_2, C[i:, i]))
        vec_C = np.hstack((vec_C, vec_C_2))
    return vec_C


# NO INTERVENTION
# ------- matrix A -------
def no_intervention_A(cov: np.ndarray, p: int) -> np.ndarray:
    return create_A_Sigma(cov, cov, p, 0)
# ------- vector c -------
def no_intervention_c(p: int) -> np.ndarray:
    return -create_vech_C(p, 0)


# SHIFT INTERVENTION
# ------- matrix A -------
def shift_intervention_A(cov: np.ndarray, mean_int: np.ndarray, p: int) -> np.ndarray:
    A_Sigma = create_A_Sigma(cov, cov, p, 0)
    temp = np.kron(mean_int, np.eye(p))
    return np.vstack((A_Sigma, temp))
# ------- vector c -------
def shift_intervention_c(p: int, index: int) -> np.ndarray:
    vech_C = create_vech_C(p, index)
    unit_vector = np.eye(p)[index]
    return np.hstack((-vech_C, shift_param * unit_vector))


# HARD INTERVENTION
# ------- matrix A -------
def hard_intervention_A(cov: np.ndarray, cov_int: np.ndarray, mean_int: np.ndarray, p: int, index: int) -> np.ndarray:
    A_Sigma = create_A_Sigma(cov=cov, cov_int=cov_int, p=p, index=index, type="hard")
    mean_int = np.insert(arr=mean_int, obj=index, values=hard_param, axis=0)
    identity_pseudo = np.eye(p)
    identity_pseudo[index, index] = 0
    temp = np.kron(mean_int, identity_pseudo)
    return np.vstack((A_Sigma, temp))
# ------- vector c -------
def hard_intervention_c(p: int, index: int) -> np.ndarray:
    vech_C = create_vech_C(p, index, type="hard")
    return np.hstack((-vech_C, np.zeros(p)))



num_exp = 10000 # number of true matrices per case
volatility_param = 2
shift_param = 2
hard_param = 2
reg_param = [0.1, 0.01, 0.001]

## Experiments

### Ex. 0
<img src="Graphics/graph40.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.

### Ex. 1
<img src="Graphics/graph41.png" alt="Drawing" style="width: 150px;"/>

In [2]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[0, 2], [1, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 7, 8])
supp_compl = np.array([2, 5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_1 = ex_nonsimple_1.copy()
ex_nonsimple_shift_1 = ex_nonsimple_1.copy()

for row in ex_nonsimple_1.itertuples():
    row = row.Index
    true_M = ex_nonsimple_1.iloc[row, 0]

    intervention_index = ex_nonsimple_1.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_1.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_1.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_1.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_1 = ex_nonsimple_1[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_1 = ex_nonsimple_hard_1[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_1 = ex_nonsimple_shift_1[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_1.to_pickle("Experiments/nonsimple_1.pkl")
ex_nonsimple_hard_1.to_pickle("Experiments/nonsimple_hard_1.pkl")
ex_nonsimple_shift_1.to_pickle("Experiments/nonsimple_shift_1.pkl")

### Ex. 2
<img src="Graphics/graph42.png" alt="Drawing" style="width: 150px;"/>

In [3]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[0, 2], [2, 0], [2, 1]])
support = np.array([0, 1, 3, 4, 5, 8])
supp_compl = np.array([2, 6, 7])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_2 = ex_nonsimple_2.copy()
ex_nonsimple_shift_2 = ex_nonsimple_2.copy()

for row in ex_nonsimple_2.itertuples():
    row = row.Index
    true_M = ex_nonsimple_2.iloc[row, 0]

    intervention_index = ex_nonsimple_2.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_2.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_2.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_2.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_2 = ex_nonsimple_2[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_2 = ex_nonsimple_hard_2[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_2 = ex_nonsimple_shift_2[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_2.to_pickle("Experiments/nonsimple_2.pkl")
ex_nonsimple_hard_2.to_pickle("Experiments/nonsimple_hard_2.pkl")
ex_nonsimple_shift_2.to_pickle("Experiments/nonsimple_shift_2.pkl")

### Ex. 3
<img src="Graphics/graph43.png" alt="Drawing" style="width: 150px;"/>

In [4]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[0, 2], [1, 2]])
support = np.array([0, 1, 3, 4, 6, 7, 8])
supp_compl = np.array([2, 5])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_3 = ex_nonsimple_3.copy()
ex_nonsimple_shift_3 = ex_nonsimple_3.copy()

for row in ex_nonsimple_3.itertuples():
    row = row.Index
    true_M = ex_nonsimple_3.iloc[row, 0]

    intervention_index = ex_nonsimple_3.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_3.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_3.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_3.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_3 = ex_nonsimple_3[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_3 = ex_nonsimple_hard_3[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_3 = ex_nonsimple_shift_3[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_3.to_pickle("Experiments/nonsimple_3.pkl")
ex_nonsimple_hard_3.to_pickle("Experiments/nonsimple_hard_3.pkl")
ex_nonsimple_shift_3.to_pickle("Experiments/nonsimple_shift_3.pkl")

### Ex. 4
<img src="Graphics/graph44.png" alt="Drawing" style="width: 150px;"/>

In [5]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[1, 2], [2, 0]])
support = np.array([0, 1, 2, 3, 4, 7, 8])
supp_compl = np.array([5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_4 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_4 = ex_nonsimple_4.copy()
ex_nonsimple_shift_4 = ex_nonsimple_4.copy()

for row in ex_nonsimple_4.itertuples():
    row = row.Index
    true_M = ex_nonsimple_4.iloc[row, 0]

    intervention_index = ex_nonsimple_4.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_4.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_4.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_4.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_4 = ex_nonsimple_4[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_4 = ex_nonsimple_hard_4[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_4 = ex_nonsimple_shift_4[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_4.to_pickle("Experiments/nonsimple_4.pkl")
ex_nonsimple_hard_4.to_pickle("Experiments/nonsimple_hard_4.pkl")
ex_nonsimple_shift_4.to_pickle("Experiments/nonsimple_shift_4.pkl")

### Ex. 5
<img src="Graphics/graph45.png" alt="Drawing" style="width: 150px;"/>

In [6]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[2, 0], [2, 1]])
support = np.array([0, 1, 2, 3, 4, 5, 8])
supp_compl = np.array([6, 7])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_5 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_5 = ex_nonsimple_5.copy()
ex_nonsimple_shift_5 = ex_nonsimple_5.copy()

for row in ex_nonsimple_5.itertuples():
    row = row.Index
    true_M = ex_nonsimple_5.iloc[row, 0]

    intervention_index = ex_nonsimple_5.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_5.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_5.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_5.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_5 = ex_nonsimple_5[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_5 = ex_nonsimple_hard_5[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_5 = ex_nonsimple_shift_5[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_5.to_pickle("Experiments/nonsimple_5.pkl")
ex_nonsimple_hard_5.to_pickle("Experiments/nonsimple_hard_5.pkl")
ex_nonsimple_shift_5.to_pickle("Experiments/nonsimple_shift_5.pkl")

### Ex. 6
<img src="Graphics/graph46.png" alt="Drawing" style="width: 150px;"/>

In [7]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[0, 2], [2, 0]])
support = np.array([0, 1, 3, 4, 5, 7, 8])
supp_compl = np.array([2, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_6 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_6 = ex_nonsimple_6.copy()
ex_nonsimple_shift_6 = ex_nonsimple_6.copy()

for row in ex_nonsimple_6.itertuples():
    row = row.Index
    true_M = ex_nonsimple_6.iloc[row, 0]

    intervention_index = ex_nonsimple_6.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_6.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_6.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_6.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_6 = ex_nonsimple_6[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_6 = ex_nonsimple_hard_6[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_6 = ex_nonsimple_shift_6[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_6.to_pickle("Experiments/nonsimple_6.pkl")
ex_nonsimple_hard_6.to_pickle("Experiments/nonsimple_hard_6.pkl")
ex_nonsimple_shift_6.to_pickle("Experiments/nonsimple_shift_6.pkl")

### Ex. 7
<img src="Graphics/graph47.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 3
seed = 0
index_true_M = 0

index_zero = np.array([[0, 2]])
support = np.array([0, 1, 3, 4, 5, 6, 7, 8])
supp_compl = np.array([2])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)] 

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "p=0.1", "p=0.01", "p=0.001"])

while index_true_M < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index_true_M, 0] = M
    index_true_M += 1

ex_nonsimple_7 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_nonsimple_hard_7 = ex_nonsimple_7.copy()
ex_nonsimple_shift_7 = ex_nonsimple_7.copy()

for row in ex_nonsimple_7.itertuples():
    row = row.Index
    true_M = ex_nonsimple_7.iloc[row, 0]

    intervention_index = ex_nonsimple_7.iloc[row, 4]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    no_A = no_intervention_A(cov=true_cov, p=p)
    no_c = no_intervention_c(p=p)
    hard_A = hard_intervention_A(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)
    hard_c = hard_intervention_c(p=p, index=intervention_index)
    shift_A = shift_intervention_A(cov=true_cov, mean_int=true_mean_shift, p=p)
    shift_c = shift_intervention_c(p=p, index=intervention_index)

    for i, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)

        direct_lasso.fit(X=no_A, y=no_c)
        M_opt = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=hard_A, y=hard_c)
        M_opt_hard = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        direct_lasso.fit(X=shift_A, y=shift_c)
        M_opt_shift = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))

        ex_nonsimple_7.iloc[row, i + 1] = np.linalg.norm(M_opt - true_M)
        ex_nonsimple_hard_7.iloc[row, i + 1] = np.linalg.norm(M_opt_hard - true_M)
        ex_nonsimple_shift_7.iloc[row, i + 1] = np.linalg.norm(M_opt_shift - true_M)

ex_nonsimple_7 = ex_nonsimple_7[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_hard_7 = ex_nonsimple_hard_7[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_shift_7 = ex_nonsimple_shift_7[["Intervention", "True M", "p=0.1", "p=0.01", "p=0.001"]]
ex_nonsimple_7.to_pickle("Experiments/nonsimple_7.pkl")
ex_nonsimple_hard_7.to_pickle("Experiments/nonsimple_hard_7.pkl")
ex_nonsimple_shift_7.to_pickle("Experiments/nonsimple_shift_7.pkl")

### Ex. 0
<img src="Graphics/graph48.png" alt="Drawing" style="width: 150px;"/>

-> Not logical since lasso promotes sparse matrix.